# Import necessary modules

In [5]:

# coding: utf-8
__author__ = "Martin Mbaya"
from __future__ import print_function
import keras
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, InputLayer, Activation, Dropout
from keras.optimizers import RMSprop, Adam, SGD
# from keras.utils import np_utils

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
from keras import backend as K
from sklearn.model_selection import train_test_split

# Data loading and preprocessing

Now we create a function that opens the csv file "prima_indians.csv". The labels are separated from the variables then normalized by subtracting the mean from each value.
The data is then split into training and testing portions to be used by the Neural Network.

In [6]:
def data():
    data_set = pd.read_csv("prima_indians.csv")
    # print(data_set.head())
    # print(data_set.columns)
    y = data_set.pop(" Class")
    x = data_set
    x -= np.mean(x, axis = 0)
    # print(x.head())
    # print(x.columns)

    # y = data_set.pop(" Class")
    # x = data_set
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
    # print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
    X_train = np.array(x_train).astype('float32')
    X_test = np.array(x_test).astype('float32')
    Y_train = keras.utils.to_categorical(y_train, 2)
    Y_test = keras.utils.to_categorical(y_test, 2)
    # print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
    return X_train, Y_train, X_test, Y_test


# Create a model

A two-layer Neural Network is then created below.
Hyperparameter tuning is done using the module called Hyperas through enclosing options for each of the parameters in "{{ }}".

In [7]:
def model(X_train, Y_train, X_test, Y_test):
    NB_EPOCH = 100
    # BATCH_SIZE = 512
    VERBOSE = 0
    NB_CLASSES = 2   # number of outputs = number of digits
    OPTIMIZER = 'RMSprop'
    RESHAPED = len(x_test.columns)

    model = Sequential()
    model.add(Dense({{choice([256, 512, 1024, 2048])}}, input_dim=RESHAPED, kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2({{uniform(0,1)}})))
    model.add(Activation({{choice(['sigmoid', 'relu'])}}))
    # model.add(Activation('sigmoid'))
#     model.add(Dropout({{uniform(0,1)}}))
    model.add(Dense(2)) 
    model.add(Activation({{choice(['softmax', 'linear'])}}))
    # model.add(Activation('softmax'))
    # model.summary() 
    model.compile(loss={{choice(['categorical_crossentropy', 'mse'])}} ,optimizer={{choice(['RMSprop', 'Adam', 'sgd'])}}, metrics=['accuracy']) 
    # model.compile(loss='mse' ,optimizer=OPTIMIZER, metrics=['accuracy'])
    
    history = model.fit(X_train, Y_train,
                        batch_size={{choice([64,128,512,1024])}},
                        epochs=NB_EPOCH, #EPOCHS
                        verbose=0,
                        validation_data=(X_test, Y_test))
                        #callbacks=[tfcall]) #For Tensorboard
    score = model.evaluate(X_test, Y_test, verbose=0)
    # print('Test loss:', score[0])
    # print('Test accuracy:', score[1])
#     print("Model run complete..")
    return {'loss': -score[1], 'status': STATUS_OK, 'model': model}


# Hyperparameter tuning

Hyperparameter tuning of number of neurons in the first layer, float value used by the l2 regularizer, activation functions, loss functions, optimizer and batch size is done using the Hyperas module.
The Tree-structured Parzen Estimator (TPE) algorithm, which explores intelligently the search space while narrowing down to the estimated best parameters is used for optimization.
The optimization is repeated over and over again, while keeping only the highest score until it either breaks the 85% mark or no improvement is observed over 10 iterations. At this point, the best overall model is selected, saved and its score printed.

In [8]:
# if __name__ == '__main__':
highest_score = [0.0, 0.0]
current_score = [0.0, 0.0]
while True:
    if highest_score[1] < 0.85:
        X_train, Y_train, X_test, Y_test = data()

        best_run, best_model = optim.minimize(model=model,
                                                data=data,
                                                algo=tpe.suggest,
                                                max_evals=10,
                                                verbose=False,
                                                trials=Trials(),
                                              notebook_name='Assignment 2 - Hyperas')

        current_score = best_model.evaluate(X_test, Y_test)
        if current_score[1] > highest_score[1]:
            model_json = best_model.to_json()
            open('prima_indians.json', 'w').write(model_json)
            best_model.save_weights('prima_indians.h5', overwrite=True)
            highest_score = current_score
            fail_count = 0
        else:
            if fail_count > 10:
                print('Test loss:', highest_score[0])
                print('Test accuracy:', (highest_score[1]*100), " %")
                break
            fail_count += 1
    else:
        print("Evaluation of best performing model: ")
        # print(highest_score, "\n")
        print('Test loss:', highest_score[0])
        print('Test accuracy:', (highest_score[1]*100), " %")
        break


231/231 [==============================] - 0s 198us/step
Test loss: 0.9810078814948276
Test accuracy: 84.8484849000906  %
